# Imports and data uploads

As is customary, let us first call the Python libraries needed here, and upload the needed data and code.

In [ ]:
from model import setup, balance_calcs, func_FDC, visuals, performance
import datetime
import pandas as pd
import math
from scipy import special
import matplotlib.pyplot as plt

# Part 1: Historical water balance

In this tutorial we will build on the model we developed in Tutorial 2.

The historical water balance will enable us to compare with results under climate change.

In [ ]:
# Preparing the model
reservoir_name = 'Conowingo'
downstream_demand_names = ['Environmental']
direct_demand_names = ['Baltimore', 'Chester', 'Nuclear plant']

# Loading the model!
conowingo = setup.define_reservoir(reservoir_name, downstream_demand_names, direct_demand_names)

# Checking on the structure, e.g.:
print('Demand downstream of the dam is ' + conowingo.demand_downstream[0].name + '.')
print('Dead storage is ' + "{:.1f}".format(conowingo.dead_storage / 100**3) + ' hm3')

In [ ]:
# Read flow and demand data. We keep this copy of the data for the simulation of different futures.
flows_default = setup.extract_flows(reservoir=conowingo)
display(flows_default)

In [ ]:
# First, make a copy of the flows to initialise the water balance
historical_balance = flows_default.copy()  # Keep flows_default as an untouched copy

# Computing the water balance for our standard operating policy (SOP)
balance_calcs.sop_full(reservoir=conowingo, water_flows=historical_balance)

# Results check
print(historical_balance.columns)

Now that we have our water balance, we can have the same visuals as before. **In particular let's focus on the dry period in the 1960s we identified in previous tutorials.**

In [ ]:
# Storage in the dry period in the 1960s
fig = visuals.storage_timeseries(reservoir=conowingo, 
                                 balance=historical_balance, 
                                 first_date=datetime.date(1962, 1, 1), 
                                 last_date=datetime.date(1968, 1, 1))

#  Part 2: A simple model to simulate a drier future

Now we want to represent a drier future. For this, **the simplest way is to multiply all flows by the same factor**. Here we uniformly decrease all flows by 20%. This means there is 20% less water on average, but also that the standard deviation of flow decreases by 20%.

In [ ]:
# First, we define our uniform model for inflows and perform the water balance with it.
def uniform_change_model(flows_original, multiplier):
    """
    This function initialises the water balance with inflows modified by the desired streamflow multiplier.
    Arguments:
        - flows_original: the flows DataFrame from reading the data. This copy is kept unmodified.
        - multiplier: float, a factor by which to multiply all flows.
    """

    # Get a copy of the data so that there is an untouched original copy
    water_balance = flows_original.copy()
    water_balance['Total inflows (m3/s)'] = water_balance['Total inflows (m3/s)'] * multiplier

    return water_balance

In [ ]:
# Define flow multiplier
flow_multiplier = 0.8  # 0.8 corresponds to a uniform 20% flow reduction

In [ ]:
# Define uniform flow reduction scenario
drier_future = uniform_change_model(flows_default, flow_multiplier)

# Perform water balance under standard operating policy (SOP)
balance_calcs.sop_full(reservoir=conowingo, water_flows=drier_future)

Now let's compare this drier future with historical data. First let's examine what that means for inflows, then for reservoir operations and performance.

## 2.1 - Compare monthly average  flows

In [ ]:
fig = visuals.compare_monthly_averages(reference=pd.Series(historical_balance['Total inflows (m3/s)']), 
                                       alternative=pd.Series(drier_future['Total inflows (m3/s)']), 
                                       labels=['Historical', 'Drier future'])

## 2.2 -  Compare daily inflows

In [ ]:
# Compare inflows especially in drier period 1962-1968
fig = visuals.compare_flow_timeseries(reference=pd.Series(historical_balance['Total inflows (m3/s)']),
                                      alternative=pd.Series(drier_future['Total inflows (m3/s)']),
                                      labels=['Historical', 'Drier future'], 
                                      first_date=datetime.date(1962, 1, 1), 
                                      last_date=datetime.date(1963, 1, 1))

## 2.3 -  Compare storage

In [ ]:
fig = visuals.compare_storage_timeseries(reservoir=conowingo, 
                                         storage_1=pd.Series(historical_balance['Storage (m3)']),
                                         storage_2=pd.Series(drier_future['Storage (m3)']), 
                                         labels=['Historical', 'Drier future'],
                                         first_date=datetime.date(1962, 1, 1), 
                                         last_date=datetime.date(1970, 1, 1))

>* **Question 1. Is a uniform flow reduction consistent with our lived experience of climate change, in particular, is it consistent with what we see in terms of extreme water events across the world?**



# Part 3: Amplified Extremes (AE) model for dry futures

As we move forward, **our objective is to generate future flow scenarios that go beyond modelling just a change in the mean. For this we use a statistical model for flow generation. We use this to create another scenario with amplified extremes.** 

In this amplified extremes (AE) model, we can adjust three parameters: mean flow, standard deviation and a the first or fifth percentile of flows (to represent how changes can particularly affect the extremes). By modifying additional parameters, we aim to capture a more nuanced representation of future flow dynamics. A full description of the model can be found [at this link](https://hess.copernicus.org/articles/27/2499/2023/). 

## 3.1 - Define a nuanced drier future flow scenario

We want to define a scenario a 20% reduction in mean flow as before, but instead of this reduction being uniform on high and low flows, we want to represent that:
>* Overall variability increases;
>* Low flows might be more severely affected than average conditions.

In [ ]:
# Define multipler of streamflow statistics for the amplified extremes (AE) model
# Change in the mean, multiplier smaller than 1 for a drier future
multiplier_mean = flow_multiplier  # For the sake of comparison with the uniform model
# Change in variability, multiplier higher than 1 for increased variability
multiplier_std = 1.5
# Change in the low flows, multiplier smaller than 1 for more severe dry conditions
multiplier_lowflow = 0.6

multipliers_AE = [multiplier_mean, multiplier_std, multiplier_lowflow]

# Executing the Model with historical flow data and a defined scenario 
drier_future_AE = balance_calcs.amplified_extremes_model(flows_original=flows_default, 
                                                         model_multipliers=multipliers_AE, 
                                                         low_quantile=1)

# Computing the water balance for our standard operating policy (SOP)
balance_calcs.sop_full(reservoir=conowingo, water_flows=drier_future_AE)

# Part 4: Let us compare all three scenarios

In this part of this tutorial, we execute the water balance model for all the scenarios. This entails comparing storage and performance metrics. This analysis will provide insights into the system's behavior under different conditions.

## 4.1 - Compare flow duration curves

A flow duration curve (FDC) plots the exceedance probability of different flow rates. This enables us to get a good idea of the flow regime, and to compare flow regimes under different climates.

In [ ]:
fig = visuals.compare_fdc(reference=pd.Series(historical_balance['Total inflows (m3/s)']), 
                          alternative=pd.Series(drier_future['Total inflows (m3/s)']), 
                          alternative_2=pd.Series(drier_future_AE['Total inflows (m3/s)']), 
                          labels=['Historical', 'Drier future (uniform)', 'Drier future (AE)'])

>* **Question 2. How do the two proposed models affect flow duration curves?**

## 4.2 - Compare monthly average flows

In [ ]:
fig = visuals.compare_monthly_averages(reference=pd.Series(historical_balance['Total inflows (m3/s)']), 
                                       alternative=pd.Series(drier_future['Total inflows (m3/s)']), 
                                       alternative_2=pd.Series(drier_future_AE['Total inflows (m3/s)']), 
                                       labels=['Historical', 'Drier future (uniform)', 'Drier future (AE)'])
fig.savefig('flows.png')

>* **Question 3. How do the two proposed models affect flow seasonality?**


## 4.3 - Compare daily flows

In [ ]:
# Compare inflows, especially in drier period 1962-1968.
fig = visuals.compare_flow_timeseries(reference=pd.Series(historical_balance['Total inflows (m3/s)']),
                                      alternative=pd.Series(drier_future['Total inflows (m3/s)']),
                                      alternative_2=pd.Series(drier_future_AE['Total inflows (m3/s)']),
                                      labels=['Historical', 'Drier future (uniform)', 'Drier future (AE)'], 
                                      first_date=datetime.date(1962, 1, 1), 
                                      last_date=datetime.date(1963, 1, 1))

>* **Question 4. How are the differences between the two models represented for daily inflows?**
>* **Question 5. What drawbacks do you see for both methods for generating futures?**




## 4.4 - Compare storage

In [ ]:
fig = visuals.compare_storage_timeseries(reservoir=conowingo, 
                                         storage_1=pd.Series(historical_balance['Storage (m3)']),
                                         storage_2=pd.Series(drier_future['Storage (m3)']), 
                                         storage_3=pd.Series(drier_future_AE['Storage (m3)']),
                                         labels=['Historical', 'Drier future(uniform)', 'Drier future (AE)'],
                                         first_date=datetime.date(1962, 1, 1), 
                                         last_date=datetime.date(1970, 1, 1))

>* **Question 6. What are the consequences for storage of using either model?**


## 4.5 - Performance comparison

All the performance code is also available in `model/performance.py` which can be imported using `from model import performance`.

We display one above the other the full metrics for:
1) Historical conditions.
2) Model 1, uniform 20% inflow decrease.
3) Model 2, 20% average inflow decrease with extreme amplification.

In [ ]:
metrics_h = performance.all_metrics(conowingo, historical_balance)
display(metrics_h)  # 1 - Historical conditions

In [ ]:
metrics_u = performance.all_metrics(conowingo, drier_future)
display(metrics_u)  # 2 - Uniform model, 20% inflow decrease.

In [ ]:
metrics_ae = performance.all_metrics(conowingo, drier_future_AE)
display(metrics_ae)  # 3 - AE model, 20% average inflow decrease with extreme amplification.

>* **Question 7. How did the choice of model affect performance?**
>* **Question 8. Try changing parameters of the AE model. For instance, what happens to performance if instead of a 20% decrease, the mean were unchanged compared with historical conditions?**

**Hint:** you can change the parameters of the AE model to better appreciate the importance of modelling changes in the extremes. For instance, if you change `multiplier_mean` to `1.0`, you can appreciate the difference between this future and the historical flows (for the same average flows).